In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from Data import*

bom_df = pd.read_excel("Bill of Material (mrp.bom)updated.xlsx")
stock_df = pd.read_excel("Reordering Rule (stock.warehouse.orderpoint).xlsx")
Stockproducts = list(stock_df['Product'])
#Machines_df = pd.read_excel("Maintenance Equipment.xlsx")
#Machines =list(Machines_df['']
Machines = ['M3-01 - Accuwell - 3axis - Manual (FR3_01)','M5-03 - Haas - 5axis - Manual (UMC750)','M3-02 - Accuwell - 3axis - Manual (FR3_02)','M3-03 - Haas - 3axis - Manual (VF2SS)']
AutomatedMachines = ['M5-04 - Haas - 5axis  - Manual (UMC750SS)','M5-01 - Accuwell - 5axis - Pallet automaton (FR5_01)','M4-01 - Accuwell -  4axis - Conveyor automation (FR4_01)','M4-02 - Accuwell - 4axis - Conveyor automation (FR4_02)','M5-02 - Accuwell - 5axis - Conveyor automation (FR5_02)']


DataMgr = DataManager()
DataMgr.getResources().clear()
DataMgr.getProducts().clear()

def MakeProduct (dbmgr,product,stock):

    if (product.find("[")>-1) and (product.find("]")>-1):
        productnumber = product[product.find("[")+1:product.find("]")];
        productname = product[product.find("]")+1:];
        if not "["+productnumber+"]_"+productname in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),"["+productnumber+"]_"+productname,productnumber,stock);
            return newproduct
        else:
            return dbmgr.getProducts()["["+productnumber+"]_"+productname]
    else:
        if not product in dbmgr.getProducts():
            newproduct = Product(len(dbmgr.getProducts()),product,product,stock);
            return newproduct
        else:
            return dbmgr.getProducts()[product]

def MakeOperation(dbmgr,operation,prod,resource,duration):
    myresource=None;
    if not resource in dbmgr.getResources():
        if resource in Machines or resource in AutomatedMachines:
            if resource in Machines:
              myresource = Resource(len(dbmgr.getResources()),"Machine",resource,15);
              myresource.setAutomated(False);
              myresource.setOperatingEffort(0.3);
              myresource.setAvailableShift("All");
              dbmgr.getResources()[resource]=myresource;
            else:
               myresource = Resource(len(dbmgr.getResources()),"Machine",resource,15);
               dbmgr.getResources()[resource]=myresource;
               myresource.setAutomated(True);
               myresource.setAvailableShift("All");
               myresource.setOperatingEffort(0.1);
        elif "Outsourced" in resource.split():
            myresource = Resource(len(dbmgr.getResources()),"Outsourced",resource,10000);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
        else:
            myresource = Resource(len(dbmgr.getResources()),"Manual",resource,15);
            myresource.setOperatingEffort(1);
            myresource.setAvailableShift("All");
            dbmgr.getResources()[resource]=myresource;
    else:
        myresource = dbmgr.getResources()[resource]; 

    
    newopr = Operation(len(dbmgr.getOperations()),operation+"_"+prod.getName(),duration)

    if myresource !=None:
        myresource.getOperations().append(newopr)
        newopr.getRequiredResources().append(myresource)

    if not newopr.getName() in dbmgr.getOperations():
        dbmgr.getOperations()[newopr.getName()]=newopr;
        
    return newopr

def SaveInstance(DataMgr,file):

        # Save Products.
        products_df = pd.DataFrame(columns= ["ProductID", "ProductNumber","Name","StockLevel"])
        precedences_df = pd.DataFrame(columns= ["PredecessorID","SuccessorID","Multiplier"])
        prodrops_df = pd.DataFrame(columns= ["ProductID","OperationID","OperationIndex"])
        operations_df = pd.DataFrame(columns= ["OperationID","Name","ProcessTime"])
        opsres_df = pd.DataFrame(columns= ["OperationID","ResourceID"])
        resources_df = pd.DataFrame(columns= ["ResourceID","ResourceType","Name","DailyCapacity","Automated","OperatingEffort","Shift"])
        orders_df = pd.DataFrame(columns= ["OrderID","ProductID","ProductName","Name","Quantity","Deadline"])
 
        
        for name,myprod in DataMgr.Products.items():
            products_df.loc[len(products_df)] = {"ProductID":myprod.getID(), "ProductNumber":myprod.getPN(),"Name":myprod.getName(),"StockLevel":myprod.getStockLevel()}

            
            for pred in myprod.getPredecessors():
                precedences_df.loc[len(precedences_df)] = {"PredecessorID":pred.getID(),"SuccessorID":myprod.getID(),"Multiplier":myprod.getMPredecessors()[pred]}
 
            oprind = 0
            for opr in myprod.getOperations(): 
                prodrops_df.loc[len(prodrops_df)] = {"ProductID":myprod.getID(),"OperationID":opr.getID(),"OperationIndex":oprind}
                oprind+=1 

        for opname,opr in DataMgr.Operations.items():
            operations_df.loc[len(operations_df)]= {"OperationID":opr.getID(),"Name":opr.getName(),"ProcessTime":opr.getProcessTime()}
            
            for res in opr.getRequiredResources(): 
                opsres_df.loc[len(opsres_df)]= {"OperationID":opr.getID(),"ResourceID":res.getID()}
        

        for resname,res in DataMgr.Resources.items():
            resources_df.loc[len(resources_df)] ={"ResourceID":res.getID(),"ResourceType":res.getType(),"Name":res.getName(),"DailyCapacity":res.getDailyCapacity(),"Automated":res.getAutomated(),"OperatingEffort":res.getOperatingEffort(),"Shift":res.getAvailableShift()}
 
        
        for ordname,ordr in DataMgr.CustomerOrders.items():
            orders_df.loc[len(orders_df)]={"OrderID":ordr.getID(),"ProductID":ordr.getProduct().getID(),"ProductName":ordr.getProduct().getName(),"Name":ordr.getName(),"Quantity":ordr.getQuantity(),"Deadline":ordr.getDeadLine()}

    


        folder = 'UseCases'; casename = file
        path = folder+"\\"+casename
        isExist = os.path.exists(path)

        if not isExist:
            os.makedirs(path)
       
        
        filename = 'Products.csv'; path = folder+"\\"+casename+"\\"+filename;  fullpath = os.path.join(Path.cwd(), path) 
        products_df.to_csv(path, index=False)
        filename = 'Precedences.csv';path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        precedences_df.to_csv(fullpath, index=False)
        filename = 'ProductsOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        prodrops_df.to_csv(fullpath, index=False)
        filename = 'ResourcesOperations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        opsres_df.to_csv(fullpath, index=False)
        filename = 'Operations.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        operations_df.to_csv(fullpath, index=False)
        filename = 'Resources.csv'; path = folder+"\\"+casename+"\\"+filename; fullpath = os.path.join(Path.cwd(), path)
        resources_df.to_csv(fullpath, index=False)
        filename = 'CustomerOrders.csv'; path = folder+"\\"+casename+"\\"+filename;fullpath = os.path.join(Path.cwd(), path)
        orders_df.to_csv(fullpath, index=False)

        return

##Create operators
Op1 = Resource(len(DataMgr.getResources()),"Operator","Operator 1",8);
Op1.setAvailableShift(1);
DataMgr.getResources()["Operator 1"]=Op1;

Op2 = Resource(len(DataMgr.getResources()),"Operator","Operator 2",8);
Op2.setAvailableShift(1);
DataMgr.getResources()["Operator 2"]=Op2;

Op3 = Resource(len(DataMgr.getResources()),"Operator","Operator 3",7);
Op3.setAvailableShift(2);
DataMgr.getResources()["Operator 3"]=Op3;
##End create operators




nopass = 0;
Stockproducts = list(stock_df['Product'])
for i,r in bom_df.iterrows():

    productrow = False;
    if nopass > 0:
        nopass -=1
        continue

    if isinstance(r['Product'],str):
        if not pd.isnull(r['Product']):
            productrow = True;

    if productrow == True:
        newproduct = MakeProduct(DataMgr,r['Product'],0)
        DataMgr.getProducts()[newproduct.getName()] = newproduct;

        if not pd.isnull(r['BoM Lines/Component']):
            rawprod=r['BoM Lines/Component']
            if rawprod in Stockproducts:
                stock = stock_df['On Hand'][Stockproducts.index(rawprod)]
            else:
                stock=0

            rawprod = MakeProduct(DataMgr,r['BoM Lines/Component'],stock)
            
            quantity = r['BoM Lines/Quantity']

            newproduct.getMPredecessors()[rawprod] = quantity
            newproduct.getPredecessors().append(rawprod)
            rawprod.setSuccessor(newproduct)

            DataMgr.getProducts()[rawprod.getName()] = rawprod

        j = i
        while j < len(bom_df):
            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('Product')]) and j > i:
                break
            #print("Operation: ",bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],"-",bom_df.iloc[j,bom_df.columns.get_loc('Reference')])
            duration = bom_df['BoM Lines/Operations/Duration'][j]
            if isinstance(duration, (int, float)):
                dur = float(1/60)*float(duration) # time assumed to be minute..

            if not pd.isnull(bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')]):
                if bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')] == "OUT - Outsourced activity":
                    myopr = MakeOperation(DataMgr,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')]+"_"+bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],dur)
                    newproduct.getOperations().append(myopr)
                    
                else:
                    myopr = MakeOperation(DataMgr,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations')],newproduct,bom_df.iloc[j,bom_df.columns.get_loc('BoM Lines/Operations/Work Center')],dur)
                    newproduct.getOperations().append(myopr)
                
               
   
            j+=1 
        nopass=len(newproduct.getOperations())-1
       
        i = j-1
        #print("i",i,"j",j)
    else:
        print(">>>>>>>>>>>> No Product definition: ", isinstance(r['Product'],str),pd.isnull(r['Product']))
        print("Product value: ",r['Product'])
        print(r)

rel_path = "Sales Order Line (sale.order.line).xlsx"
abs_file_path = os.path.join(Path.cwd(), rel_path)
xls = pd.ExcelFile(abs_file_path)
orders_df = pd.read_excel(xls,"Sheet1")

#orders_df['Delivery Date']  = pd.to_datetime(orders_df['Delivery Date'] )

for i,r in orders_df.iterrows():
    code = r['Description']
    if code.find("[")>-1 and code.find("]")>-1:
        code = code[code.find("[")+1:code.find("]")]
    else:
        continue
    
    ordername = r['Order Reference']+"_("+code+")"

    prodmatch = [prod for prod in DataMgr.getProducts().values() if prod.getPN() == code]

    if len(prodmatch) == 0:
        continue

    myprod = prodmatch[0]

    deadline = datetime.today().strftime('%Y-%m-%d')

    if not pd.isnull(r['Delivery Date']):
        deadline = r['Delivery Date'].strftime('%Y-%m-%d')
    

    #print(r['Delivery Date'],pd.isnull(r['Delivery Date']))
    myorder = CustomerOrder(len(DataMgr.getCustomerOrders()),ordername,myprod.getID()
                                                           ,myprod.getName(),r['Quantity'],str(deadline))
    myorder.setProduct(myprod)

    DataMgr.getCustomerOrders()[ordername] = myorder

print('Nr Products: ',len(DataMgr.getProducts()))
print('Nr Resources: ',len(DataMgr.getResources()))
print('Nr Operations: ',len(DataMgr.getOperations()))
print("Nr Customer Orders: ",len(DataMgr.getCustomerOrders()))

# SaveInstance(DataMgr,'TBRM_Volledige_Instantie')

    

    
    
        

    

Nr Products:  561
Nr Resources:  32
Nr Operations:  984
Nr Customer Orders:  127


In [2]:
import os
online_version = False;
from DBMain import*

#############################################################################################################
tab_1 = VisMgr.get_case_selection_tab(); tab_4 = VisMgr.generatePSTAB(); tab_2 = VisMgr.generateCOTAB(); tab_3 = VisMgr.generatePLTAB()
tab_5 = VisMgr.generatePSschTAB()
####################################################################################################################
tab_set = widgets.Tab([tab_1,tab_4,tab_2,tab_3,tab_5])
tab_set.set_title(0, 'UseCase Selection'); tab_set.set_title(1, 'Production System'); tab_set.set_title(2, 'Customer Orders');
tab_set.set_title(3, 'Production Planning'); tab_set.set_title(4, 'Production Schedule');tab_set

In [3]:
a = DataMgr

In [4]:
b = a.getResources()

In [ ]:
b

In [5]:
test=b['M3-01 - Accuwell - 3axis - Manual (FR3_01)']

In [ ]:
test

In [6]:
test.getSchedule()

{<PlanningObjects.Shift at 0x1d397669d90>: [(<PlanningObjects.Job at 0x1d3987dc740>,
   0),
  (<PlanningObjects.Job at 0x1d3987dc680>, 1.3999999999999944),
  (<PlanningObjects.Job at 0x1d3987dc5c0>, 2.7999999999999887),
  (<PlanningObjects.Job at 0x1d3987dc500>, 3.999999999999984),
  (<PlanningObjects.Job at 0x1d3987dc440>, 5.399999999999978),
  (<PlanningObjects.Job at 0x1d3987dc380>, 6.799999999999972),
  (<PlanningObjects.Job at 0x1d3987fdcd0>, 7.999999999999968),
  (<PlanningObjects.Job at 0x1d3987fdc70>, 7.999999999999968)],
 <PlanningObjects.Shift at 0x1d397668b60>: [(<PlanningObjects.Job at 0x1d3987dc2c0>,
   0),
  (<PlanningObjects.Job at 0x1d3987dc200>, 1.3999999999999944),
  (<PlanningObjects.Job at 0x1d3987dc140>, 2.7999999999999887),
  (<PlanningObjects.Job at 0x1d3987dc080>, 3.999999999999984),
  (<PlanningObjects.Job at 0x1d3987cbf80>, 5.399999999999978),
  (<PlanningObjects.Job at 0x1d397713e00>, 6.799999999999972)],
 <PlanningObjects.Shift at 0x1d39766e390>: [(<Planning

In [43]:
oprdict = dict() # key: operation, #val: set of jobs
      

for resname,res in self.getDataManager().getResources().items():
    res.getSchedule().clear()

# Collect operations with jobs
nrjobs = 0
for prname,prod in self.getDataManager().getProducts().items():
    for opr in prod.getOperations():
        if not opr in oprdict:
            oprdict[opr] = opr.getJobs()
            nrjobs+= len(opr.getJobs())
            
self.getVisualManager().getPSchScheRes().value+=" To schedule jobs: "+str(nrjobs)+"\n"
revopdict = {k: oprdict[k] for k in sorted(oprdict, key=lambda x: list(oprdict.keys()).index(x), reverse=True)}


for k1, k2 in zip(revopdict, list(revopdict)[1:]): #links pairs of keys together 
    if len(revopdict[k1]) > 0 and len(revopdict[k2]) >0:
        CurJobs = revopdict[k1][::-1];
        Predjobs = revopdict[k2][::-1];
        
        for i in range(0,len(CurJobs)):
            CapCurJob = CurJobs[i].getQuantity();           
                      
            
            for k in Predjobs:
                
                if k.getQuantity() < CapCurJob:
                    CapCurJob = CapCurJob - k.getQuantity();
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)
                else:
                    if k not in CurJobs[i].getPredecessors():
                        CurJobs[i].getPredecessors().append(k);
                    
                    if CurJobs[i] not in k.getSuccessor():
                        k.getSuccessor().append(CurJobs[i]);
                    Predjobs.remove(k)                       
                    break
## Initialize shifts (example 30 days?)
shiftnum = 1000;

i=1;
day=1;
shiftlist=[]
while i <= shiftnum:
    if i % 2 == 1:
        shift = Shift(i,1,8)
        shiftlist.append(shift)
        i+=1;
    else:
        shift=Shift(i,2,7)
        shiftlist.append(shift)
        i+=1

#Initialize Schedulable Jobs
AllJobs = dict()
SchedulableJobs=[]

ScheduledJobs=[]
for opr, jobs in oprdict.items():
    for job in jobs:
        if job.getPredecessors() == []:
            SchedulableJobs.append(job)
        AllJobs[job.getName()]=job;
counter = 0       
#Initialize schedule for each Resource:
for resname, res in self.getDataManager().getResources().items():
    for i in shiftlist:
        res.getSchedule()[i]=[]

#Create Schedule
while len(SchedulableJobs) >0:
    for j in SchedulableJobs:
        predecessorjobs = j.getPredecessors()
        successorjobs = j.getSuccessor()
        if not (predecessorjobs ==[]):
        #Determine Earliest starttime
            maxpredecessor = None;
            for i in predecessorjobs:
                completiontime = i.getStartTime() + (i.getQuantity() * self.getDataManager().getOperations()[i.getOperation().getName()].getProcessTime())
                if maxpredecessor == None or completiontime >= maxpredecessor:
                    maxpredecessor = completiontime
            EarliestStart = maxpredecessor;
        else:
            EarliestStart = 0
            
        #Determine earliest available resource
        processtime = (j.getQuantity() * self.getDataManager().getOperations()[j.getOperation().getName()].getProcessTime())
        resources = j.getOperation().getRequiredResources()
        for r in resources:
            Automated = r.getAutomated()
            opef = r.getOperatingEffort()
            for shift, jobtime in r.getSchedule().items():
                
                ##Here we now have the completion time of the last job in a shift. Check if the to job to schedule fits in the shift. Else check next shift.
                if jobtime ==[]:
                    completiontimeLatestJob = 0;
                else:
                    completiontimeLatestJob = jobtime[::-1][0][1] + (jobtime[::-1][0][0].getQuantity() * self.getDataManager().getOperations()[jobtime[::-1][0][0].getOperation().getName()].getProcessTime())
                
                shiftcap = shift.getCapacity()
                shiftnumber = shift.getNumber()
                
                if shiftnumber == 1:
                    CurOpeffOp1 = 0; ## Also check operator availability.
                    SchedOp1 = self.getDataManager().getResources()['Operator 1'].getSchedule();
                    CurOpeffOp2 = 0;
                    SchedOp2 = self.getDataManager().getResources()['Operator 2'].getSchedule();
                    for jobs in SchedOp1[shift]:
                        for resOpJob in jobs[0].getOperation().getRequiredResources():
                            CurOpeffOp1 += resOpJob.getOperatingEffort()
                    for jobs in SchedOp2[shift]:
                        for resOpJob in jobs[0].getRequiredResources():
                            CurOpeffOp2 += resOpJob.getOperatingEffort()

                    if shiftcap > (completiontimeLatestJob + processtime) and (CurOpeffOp1 + opef <= 1 or CurOpeffOp2 + opef <=1): #This means we can schedule in first shift
                        j.setStartTime(completiontimeLatestJob)
                        if (CurOpeffOp1 +opef <= 1):
                           SchedOp1[shift].append((j,completiontimeLatestJob))
                        else:
                            SchedOp2[shift].append((j,completiontimeLatestJob))
                        r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                        ScheduledJobs.append(j)
                        ## check if successor can be scheduled.
                        for sucjob in successorjobs:
                            Schedulable = True
                            for predjbs in sucjob.getPredecessors():
                                if predjbs in ScheduledJobs:
                                    continue
                                else:
                                    Schedulable = False;
                            if Schedulable == True:
                                SchedulableJobs.append(sucjob)
                        
                        SchedulableJobs.remove(j) #Remove scheduled job
                        break
                    else:
                        continue

                if shiftnumber == 2:
                    CurOpeffOp3 = 0; ## Also check operator availability.
                    SchedOp3 = self.getDataManager().getResources()['Operator 3'].getSchedule();
                    
                    for jobs in SchedOp3[shift]:
                        for resOpJob in jobs[0].getOperation().getRequiredResources():
                            CurOpeffOp3 += resOpJob.getOperatingEffort()
                    if  ((Automated is None) and processtime < 8) or Automated == False:
                        if shiftcap > (completiontimeLatestJob + processtime) and (CurOpeffOp3 + opef<=1): #This means we can schedule in second shift
                            j.setStartTime(completiontimeLatestJob)
                            SchedOp3[shift].append((j,completiontimeLatestJob))
                            
                            r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                            ScheduledJobs.append(j)
                            ## check if successors can be scheduled.
                            for sucjob in successorjobs:
                                Schedulable = True
                                for predjbs in sucjob.getPredecessors():
                                    if predjbs in ScheduledJobs:
                                        continue
                                    else:
                                        Schedulable = False;
                                if Schedulable == True:
                                    SchedulableJobs.append(sucjob)
                            
                            SchedulableJobs.remove(j) #Remove scheduled job
                            break
                        else:
                            continue

                    elif (Automated == True) or (Automated is None and processtime > 8):                        
                        if shiftcap >= (completiontimeLatestJob) and (CurOpeffOp3 + opef<=1): #This means we can schedule in second shift
                            j.setStartTime(completiontimeLatestJob)
                            SchedOp3[shift].append((j,completiontimeLatestJob))
                            
                            r.getSchedule()[shift].append((j,completiontimeLatestJob))                                
                            ScheduledJobs.append(j)
                            ## check if successors can be scheduled.
                            for sucjob in successorjobs:
                                Schedulable = True
                                for predjbs in sucjob.getPredecessors():
                                    if predjbs in ScheduledJobs:
                                        continue
                                    else:
                                        Schedulable = False;
                                if Schedulable == True:
                                    SchedulableJobs.append(sucjob)
                            
                            SchedulableJobs.remove(j) #Remove scheduled job
                            break
                        else:
                            continue                        
            break
            

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [38]:
processtime

8.399999999999967

In [39]:
Automated

In [23]:
j.getPredecessors()

[]

In [25]:
processtime

8.399999999999967

In [26]:
j.getQuantity()

504

In [28]:
j.getOperation().getProcessTime()

0.0166666666666666

In [56]:
processtime

0.999999999999996

In [34]:
r.getOperatingEffort()

0

In [58]:
resources

In [59]:
resources[0].getName()

'MP1 - Material preparation - automatic sawing (ZGN_01)'

In [41]:
r.getSchedule()

{<PlanningObjects.Shift at 0x23ebf06f440>: [(<PlanningObjects.Job at 0x23ec4d35ee0>,
   0),
  (<PlanningObjects.Job at 0x23ec4d48320>, 4.800000000000001)],
 <PlanningObjects.Shift at 0x23ec28ea840>: [(<PlanningObjects.Job at 0x23ec4d35c10>,
   0)],
 <PlanningObjects.Shift at 0x23ec29314f0>: [(<PlanningObjects.Job at 0x23ec4d485f0>,
   0)],
 <PlanningObjects.Shift at 0x23ec29323c0>: [(<PlanningObjects.Job at 0x23ec4d36090>,
   0),
  (<PlanningObjects.Job at 0x23ec4d641a0>, 3.6)],
 <PlanningObjects.Shift at 0x23ec4edbb60>: [(<PlanningObjects.Job at 0x23ec4d35f70>,
   0),
  (<PlanningObjects.Job at 0x23ec4d48500>, 4.800000000000001)],
 <PlanningObjects.Shift at 0x23ec4edaff0>: [(<PlanningObjects.Job at 0x23ec4d64320>,
   0)],
 <PlanningObjects.Shift at 0x23ec4edb470>: [(<PlanningObjects.Job at 0x23ec4d36000>,
   0)],
 <PlanningObjects.Shift at 0x23ec4edb3e0>: [(<PlanningObjects.Job at 0x23ec4d643e0>,
   0)],
 <PlanningObjects.Shift at 0x23ec4edb2f0>: [(<PlanningObjects.Job at 0x23ec4d3687

In [65]:
r.getSchedule().keys()

dict_keys([<PlanningObjects.Shift object at 0x000001AD97D57650>, <PlanningObjects.Shift object at 0x000001AD95D058B0>, <PlanningObjects.Shift object at 0x000001AD97CFFAD0>, <PlanningObjects.Shift object at 0x000001AD97D7F050>, <PlanningObjects.Shift object at 0x000001AD98227EF0>, <PlanningObjects.Shift object at 0x000001AD9821FB30>, <PlanningObjects.Shift object at 0x000001AD9821FB60>, <PlanningObjects.Shift object at 0x000001AD9821F4D0>, <PlanningObjects.Shift object at 0x000001AD9821DB80>, <PlanningObjects.Shift object at 0x000001AD99214C80>, <PlanningObjects.Shift object at 0x000001AD99214A40>, <PlanningObjects.Shift object at 0x000001AD98221010>, <PlanningObjects.Shift object at 0x000001AD98221100>, <PlanningObjects.Shift object at 0x000001AD98221580>, <PlanningObjects.Shift object at 0x000001AD98221310>, <PlanningObjects.Shift object at 0x000001AD98220EF0>, <PlanningObjects.Shift object at 0x000001AD982212B0>, <PlanningObjects.Shift object at 0x000001AD98220FB0>, <PlanningObjects.

In [66]:
len(list(r.getSchedule().keys()))

30

In [68]:
CurOpEff3

NameError: name 'CurOpEff3' is not defined